In [79]:
import seaborn as sns
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv(r"C:\Users\dagma\Documents\project\TB_Burden_Country.csv")
df.head

<bound method NDFrame.head of      Country or territory name ISO 2-character country/territory code  \
0                  Afghanistan                                     AF   
1                  Afghanistan                                     AF   
2                  Afghanistan                                     AF   
3                  Afghanistan                                     AF   
4                  Afghanistan                                     AF   
...                        ...                                    ...   
5115                  Zimbabwe                                     ZW   
5116                  Zimbabwe                                     ZW   
5117                  Zimbabwe                                     ZW   
5118                  Zimbabwe                                     ZW   
5119                  Zimbabwe                                     ZW   

     ISO 3-character country/territory code  \
0                                       AFG   

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5120 entries, 0 to 5119
Data columns (total 47 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   Country or territory name                                                                       5120 non-null   object 
 1   ISO 2-character country/territory code                                                          5096 non-null   object 
 2   ISO 3-character country/territory code                                                          5120 non-null   object 
 3   ISO numeric country/territory code                                                              5120 non-null   int64  
 4   Region                                                                                          5120 non-null   object 
 5   Year         

In [82]:
year_cols = df['Year']
print("Number of year columns:", len(year_cols))

Number of year columns: 5120


TO check if all the years are in long format and not in wide format

In [83]:
columns_to_keep = [
    "Country or territory name",
    "Region",
    "Year",
    "Estimated total population number",
    "Estimated number of incident cases (all forms)",
    "Estimated incidence (all forms) per 100 000 population",
    "Estimated number of deaths from TB (all forms, excluding HIV)",
    "Case detection rate (all forms), percent",
    "Estimated prevalence of TB (all forms)" 
]
data_clean = df[columns_to_keep].copy()

In [84]:
data_clean.drop_duplicates(inplace=True)

In [85]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5120 entries, 0 to 5119
Data columns (total 9 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Country or territory name                                      5120 non-null   object 
 1   Region                                                         5120 non-null   object 
 2   Year                                                           5120 non-null   int64  
 3   Estimated total population number                              5120 non-null   int64  
 4   Estimated number of incident cases (all forms)                 5120 non-null   float64
 5   Estimated incidence (all forms) per 100 000 population         5120 non-null   float64
 6   Estimated number of deaths from TB (all forms, excluding HIV)  5120 non-null   float64
 7   Case detection rate (all forms), percent                     

In [86]:
data_clean.head()

,Country or territory name,Region,Year,Estimated total population number,Estimated number of incident cases (all forms),Estimated incidence (all forms) per 100 000 population,"Estimated number of deaths from TB (all forms, excluding HIV)","Case detection rate (all forms), percent",Estimated prevalence of TB (all forms)
0,Afghanistan,EMR,1990,11731193,22000.0,189.0,4300.0,20.0,36000.0
1,Afghanistan,EMR,1991,12612043,24000.0,191.0,5800.0,96.0,43000.0
2,Afghanistan,EMR,1992,13811876,26000.0,191.0,7400.0,NaN,51000.0
3,Afghanistan,EMR,1993,15175325,29000.0,189.0,9100.0,NaN,59000.0
4,Afghanistan,EMR,1994,16485018,31000.0,188.0,11000.0,NaN,68000.0


In [87]:
print(data_clean.isnull().sum()) 

Country or territory name                                          0
Region                                                             0
Year                                                               0
Estimated total population number                                  0
Estimated number of incident cases (all forms)                     0
Estimated incidence (all forms) per 100 000 population             0
Estimated number of deaths from TB (all forms, excluding HIV)      0
Case detection rate (all forms), percent                         449
Estimated prevalence of TB (all forms)                             0
dtype: int64


In [88]:
data_clean = data_clean[data_clean["Case detection rate (all forms), percent"].notna()]

Here we used this code to show Cases where Detection Rate is not blank because it is a clinical data filling it with mean and other values is not a good choice as it can distort medical analysis

In [89]:
print(data_clean.isnull().sum()) 

Country or territory name                                        0
Region                                                           0
Year                                                             0
Estimated total population number                                0
Estimated number of incident cases (all forms)                   0
Estimated incidence (all forms) per 100 000 population           0
Estimated number of deaths from TB (all forms, excluding HIV)    0
Case detection rate (all forms), percent                         0
Estimated prevalence of TB (all forms)                           0
dtype: int64


In [90]:
data_clean["Year"] = data_clean["Year"].astype(int)

Here we changed the year into an int data type

In [91]:
data_clean.rename(columns={
    'Country or territory name': 'Country',
    'Estimated number of incident cases (all forms)': 'Total_Cases',
    'Estimated incidence (all forms) per 100 000 population': 'Incidence_Rate',
    'Estimated total population number': 'Population',
    "Estimated total population number": "Total Population",
    "Estimated number of incident cases (all forms)": "Total New TB Cases",
    "Case detection rate (all forms), percent": "Case Detection Rate (%)",
    "Estimated prevalence of TB (all forms)": "People Living with TB"
}, inplace=True)

Renaming Varibales for better understanding

In [92]:
region_mapping = {
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean',
    'EUR': 'Europe',
    'SEA': 'South-East Asia',
    'WPR': 'Western Pacific'
}

data_clean['Region'] = data_clean['Region'].map(region_mapping)

print(df['Region'].unique())

['EMR' 'EUR' 'AFR' 'WPR' 'AMR' 'SEA']


Mapped the region name from abbreviation to their full name for better understanding when applying filters 

In [93]:
data_clean["Cases_per_100k"] = (data_clean["Total New TB Cases"] / data_clean["Total Population"]) * 100000

So in this step we created a new variable called Cases_per_100k, a country with 1 million people and 10,000 cases is very different from a country with 100 million people and 10,000 cases therfore By standardizing the data per 100,000 population we will make it fairly comparable between countries and easier to visualize

In [94]:
data_clean.to_csv("TB_Variable_selected.csv", index=False)

We finshed our data cleaning steps in jupyter notebook here now we will start visualization in Microsoft power bi

In [78]:
data_clean.head()

,Country,Region,Year,Total Population,Total New TB Cases,"New TB Cases per 100,000",Total TB Deaths,"TB Deaths per 100,000",Case Detection Rate (%),People Living with TB,Cases_per_100k
0,Afghanistan,Eastern Mediterranean,1990,11731193,22000.0,189.0,4300.0,37.0,20.0,36000.0,187.534209
1,Afghanistan,Eastern Mediterranean,1991,12612043,24000.0,191.0,5800.0,46.0,96.0,43000.0,190.294308
7,Afghanistan,Eastern Mediterranean,1997,19021226,36000.0,189.0,14000.0,72.0,3.6,85000.0,189.262248
8,Afghanistan,Eastern Mediterranean,1998,19496836,37000.0,189.0,14000.0,72.0,8.3,88000.0,189.774382
9,Afghanistan,Eastern Mediterranean,1999,19987071,38000.0,190.0,14000.0,71.0,8.7,89000.0,190.122905
